In [1]:
import time
import numpy as np
from Conways import*

pygame 2.0.2 (SDL 2.0.16, Python 3.9.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env = conways(size = 10, max_it=1000)
print(env.run())

0


In [3]:
x = np.zeros(shape=(10, 10), dtype=bool)
x[1, 1] = True
x[1, 2] = True
x[2, 1] = True
x[2, 2] = True
env.reset(x, offset=False)
actions = env.run()
print(actions)

inf


In [4]:
alpha = 0.1 
max_iterations = 6000
gamma = 0.995

total = 0
tot_reward = 0
prev_reward = 0

q_table_row_limit = 75

epsilon = 1
epsilon_decay = 0.99995

In [5]:
q_table = np.random.uniform(low=0, high=1, size=[q_table_row_limit, 10, 10])
q_table.shape

print(np.argmax(q_table[0]))
print(np.max(q_table[0]))
print(q_table[0])

64
0.98532588908147
[[0.6309982  0.74848516 0.76417106 0.41582912 0.95020608 0.4964031
  0.95828464 0.42256276 0.03392095 0.56821832]
 [0.00167839 0.13498901 0.04475655 0.05793677 0.43361859 0.10677179
  0.64012982 0.30123743 0.74260059 0.73662696]
 [0.49009514 0.77284674 0.98365516 0.18992405 0.32751843 0.2213401
  0.64414573 0.66386402 0.81433358 0.89509584]
 [0.64306116 0.97591595 0.85874284 0.33463346 0.12988458 0.4889838
  0.3040859  0.97735692 0.2805975  0.33533257]
 [0.35919554 0.72938594 0.35298474 0.48802518 0.63633568 0.65750408
  0.71158915 0.47616752 0.84632763 0.65011769]
 [0.50028273 0.05015964 0.3855066  0.12416422 0.50131976 0.18434539
  0.86792231 0.83248545 0.35630037 0.25717447]
 [0.39022335 0.76749015 0.74831643 0.60411075 0.98532589 0.54906516
  0.43723077 0.9353215  0.17624019 0.03337187]
 [0.39245417 0.51311499 0.40089522 0.167976   0.83580197 0.82211912
  0.44504472 0.8332953  0.6765988  0.20320517]
 [0.30556427 0.19859827 0.40874091 0.00729225 0.74258645 0.6772

In [6]:

state_score = np.array([float('-inf') for i in range(q_table_row_limit)])
state_table = np.random.uniform(low=0, high=1, size=[q_table_row_limit, 10, 10]) 

def get_discrete_state(s):
    for i in range(q_table_row_limit):
        if np.array_equal(s, state_table[i]):
            return i
    i = np.argmin(state_score)
    state_table[i] = np.copy(s)
    q_table[i] = np.random.uniform(low=0, high=1, size=[10, 10])
    return i


In [7]:
current_state = np.zeros(shape=(10, 10), dtype=bool)

#take initial action
discrete_state = get_discrete_state(current_state)

action = np.argmax(q_table[discrete_state])
action = (int(action/10), action%10)
current_state[action] = True
best_state = current_state
env.reset(current_state, offset = False)
state_score[discrete_state] = env.run()
best_reward = float('-inf')

for i in range(max_iterations):
    t0 = time.time()
    

    if (i % 2000 == 0):
        print(f"Episode: {i}")

    reward = 0
    #test possible actions after taking that action
    for j in range(q_table_row_limit):
        if np.random.random() > epsilon:
            action = np.argmax(q_table[discrete_state])
        else:
            action = np.random.randint(0, 100)
        action = (int(action/10), action%10)
        current_state[action] = True
        env.reset(current_state, offset = False)

        new_discrete_state = get_discrete_state(current_state)

        state_score[discrete_state] = env.run()
        reward += state_score[discrete_state]

        if (state_score[discrete_state] > best_reward):
            best_state = np.copy(current_state)
            best_reward = state_score[discrete_state]

        max_future_q = np.max(q_table[new_discrete_state])
        current_q = q_table[(discrete_state, action)]

        new_q = (1 - alpha) * current_q + alpha * (state_score[discrete_state] + gamma * max_future_q)
        q_table[(discrete_state, action)] = new_q

        discrete_state = new_discrete_state
    
    
    current_state = np.copy(best_state)
    reward = np.max(state_score)
    if epsilon > 0.05: 
        if reward > prev_reward and i > 10000:
            epsilon = epsilon_decay ** (i - 10000)

            if i % 500 == 0:
                print("Epsilon: " + str(epsilon))
    
    t1 = time.time()
    iteration_time = t1-t0
    total += iteration_time

    tot_reward += reward
    prev_reward = reward

    if i % 1000 == 0: 
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = tot_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        tot_reward = 0

Episode: 0
Time Average: 0.00025299835205078126
Mean Reward: 0.003
Time Average: 0.2576314511299133
Mean Reward: 51.139
Episode: 2000
Time Average: 0.24571739292144776
Mean Reward: 61.0
Time Average: 0.24791902232170104
Mean Reward: 61.0
Episode: 4000
Time Average: 0.24530016016960143
Mean Reward: 61.0
Time Average: 0.24582929873466491
Mean Reward: 61.0


In [8]:
Final = best_state
print(Final)
env.reset(Final, offset = False)
env.render = True
print(env.run())

[[False  True  True False False False False False False False]
 [False False  True  True False False  True  True  True False]
 [ True False  True  True False False False False False False]
 [False False False False False False  True False False False]
 [False  True False  True False  True  True  True False False]
 [False False False  True  True False  True  True  True False]
 [ True False False False False False False False False  True]
 [ True False  True False False False False False False False]
 [False  True False  True  True False False  True False  True]
 [ True False False False False  True  True False  True False]]
121
